In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import pyvista as pv
import os
import utils
from stpyvista import stpyvista
from datetime import datetime
import importlib

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\stpyvista\__init__.py:18: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("vtk", sizing_mode="stretch_both")


In [2]:
df = pd.concat([pd.read_csv('hka-aqm-am/' + f.removeprefix('._'), skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df.shape


(608036, 18)

In [5]:
importlib.reload(utils)
performance_df = pd.read_csv('model_performances.csv')
d_model=256
nhead=8
num_layers=5 
dropout_pe=0.15
dropout_encoder=0.15
batch_size=1024
learning_rate=0.00031
epochs=25
for aggregation_level in ['quarter_hour', 'hour', 'half_hour']:
    if aggregation_level != 'quarter_hour':
        continue
    for y_feature in ['CO2', 'VOC', 'hum', 'tmp', 'vis']:
        if y_feature != 'CO2':
            continue        
        if aggregation_level != 'quarter_hour' and y_feature in ['CO2', 'VOC']:
            num_layers += 1
            dropout_pe -= 0.05
            dropout_encoder -= 0.05
        if y_feature == 'hum':
            # hum we are overfitting with the same parameters. Decreasing num_layers by 2
            num_layers -= 2
        elif y_feature == 'tmp':
            # tmp we are overfitting a little bit with the same parameters. Increasing dropout probability by 0.05 and decreasing num_layers by 1
            num_layers -= 1
            dropout_pe += 0.05
            dropout_encoder += 0.05
        
        model, scaler, rmse, mae, me, mape, train_loss, val_loss, n_features = utils.create_multivariate_transformer_model_for_feature(df, d_model=d_model, nhead=nhead, num_layers=num_layers, dropout_pe=dropout_pe, dropout_encoder=dropout_encoder, batch_size=batch_size, learning_rate=learning_rate, epochs=epochs, y_feature=y_feature, aggregation_level=aggregation_level)
        performance_df = performance_df.append({'model_name': 'multivariate_transformer','aggregation_level': aggregation_level, 'y_feature': y_feature, 'n_features': n_features, 'rmse': rmse, 'mae': mae, 'me': me, 'mape': mape, 'd_model': d_model, 'nhead': nhead, 'num_layers': num_layers, 'dropout_pe': dropout_pe,'dropout_encoder': dropout_encoder, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs, 'train_loss': train_loss, 'val_loss': val_loss, 'note': '2 decoder Linear layers with LeakyReLu, shuffled training data'}, ignore_index=True)
        performance_df.to_csv('model_performances.csv', index=False)
        
        if y_feature == 'hum':
            # reverting changes
            num_layers += 2
        if y_feature == 'tmp':
            # reverting changes
            num_layers += 1
            dropout_pe -= 0.05
            dropout_encoder -= 0.05
    
        if aggregation_level != 'quarter_hour' and y_feature in ['CO2', 'VOC']:
            num_layers -= 1
            dropout_pe += 0.05
            dropout_encoder += 0.05


training data cutoff:  2023-07-15 02:45:00


c:\Studium\Semester_6\IoT_Project\utils.py:1063: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[columns_to_scale] = scaler.transform(df_test[columns_to_scale])
c:\Studium\Semester_6\IoT_Project\utils.py:1064: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_feature_scaler_index = columns_to_scale.index(y_feature)
c:\Studium\Semester_6\IoT_Project\utils.py:1068: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pand

device_id              int8
tmp                 float64
hum                 float64
CO2                 float64
VOC                 float64
vis                 float64
IR                  float64
WIFI                float64
BLE                 float64
rssi                float64
channel_rssi        float64
channel_index       float64
spreading_factor    float64
bandwidth           float64
f_cnt               float64
isHoliday           float64
isExamTime          float64
weekday_sin         float64
weekday_cos         float64
month_sin           float64
month_cos           float64
time_sin            float64
time_cos            float64
semester_SS23       float64
semester_WS22/23    float64
semester_WS23/24    float64
group                 int32
dtype: object
Training data shape: torch.Size([244176, 20, 25]) torch.Size([244176]) torch.Size([244176, 1])
Testing data shape: torch.Size([62745, 20, 25]) torch.Size([62745]) torch.Size([62745, 1])
Shuffled Training data shape: torch.Size([24

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


          actual   predicted
0     443.000000  431.308310
1     523.000001  525.767821
2     499.000000  464.358527
3     471.000001  469.555571
4     501.000000  482.699691
...          ...         ...
1019  544.000002  534.258910
1020  408.923076  407.077108
1021  507.999999  488.037525
1022  408.999997  426.956682
1023  425.000000  418.838349

[1024 rows x 2 columns]
Epoch 1/25, Training Loss: 0.2826
Epoch 1/25, Validation Loss: 0.2234
          actual   predicted
0     443.000000  433.620658
1     523.000001  524.690336
2     499.000000  477.226972
3     471.000001  484.843846
4     501.000000  481.020445
...          ...         ...
1019  544.000002  538.900818
1020  408.923076  421.180892
1021  507.999999  493.712011
1022  408.999997  435.711744
1023  425.000000  436.534878

[1024 rows x 2 columns]
Epoch 2/25, Training Loss: 0.1745
Epoch 2/25, Validation Loss: 0.1707
          actual   predicted
0     443.000000  432.999273
1     523.000001  541.208991
2     499.000000  493.62003

In [7]:
importlib.reload(utils)
utils.create_multivariate_transformer_model_for_feature(df, d_model=d_model, nhead=nhead, num_layers=num_layers, dropout_pe=dropout_pe, dropout_encoder=dropout_encoder, batch_size=batch_size, learning_rate=learning_rate, epochs=1, y_feature="CO2", aggregation_level=aggregation_level)

training data cutoff:  2023-07-14 07:30:00
device_id              int8
tmp                 float64
hum                 float64
CO2                 float64
VOC                 float64
vis                 float64
IR                  float64
WIFI                float64
BLE                 float64
rssi                float64
channel_rssi        float64
channel_index       float64
spreading_factor    float64
bandwidth           float64
f_cnt               float64
isHoliday           float64
isExamTime          float64
weekday_sin         float64
weekday_cos         float64
month_sin           float64
month_cos           float64
time_sin            float64
time_cos            float64
semester_SS23       float64
semester_WS22/23    float64
semester_WS23/24    float64
group                 int32
dtype: object


c:\Studium\Semester_6\IoT_Project\utils.py:1066: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # drop unconvertible columns
c:\Studium\Semester_6\IoT_Project\utils.py:1067: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(['date_time_rounded'], axis=1, inplace=True)
c:\Studium\Semester_6\IoT_Project\utils.py:1071: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

Training data shape: torch.Size([152512, 20, 25]) torch.Size([152512]) torch.Size([152512, 1])
Testing data shape: torch.Size([38893, 20, 25]) torch.Size([38893]) torch.Size([38893, 1])
Shuffled Training data shape: torch.Size([153124, 20, 25]) torch.Size([153124]) torch.Size([153124, 1])
Shuffled Testing data shape: torch.Size([38281, 20, 25]) torch.Size([38281]) torch.Size([38281, 1])
26


KeyError: 'group'

In [26]:
all_model_files = [f for f in os.listdir('models/')]
all_model_files_no_version = [f.rsplit('_', 1)[0] for f in all_model_files]
all_model_files_no_version_unique = list(set(all_model_files_no_version))
all_model_files_no_version_unique
for model_file in all_model_files_no_version_unique:
    model_files = [f for f in all_model_files if f.startswith(model_file)]
    model_files.sort()
    latest_model_file = model_files[-1]
    print(latest_model_file)
    # delete all other versions
    for model_file_to_delete in model_files[:-1]:
        print('deleting ' + model_file_to_delete)
        os.remove('models/' + model_file_to_delete)
    # rename latest model file to v1
    file_ending = latest_model_file.rsplit('.', 1)[1]
    latest_model_file_rename = latest_model_file.rsplit('_', 1)[0] + '_v1' + '.' + file_ending
    if latest_model_file != latest_model_file_rename:
        print('renaming ' + latest_model_file + ' to ' + latest_model_file_rename)
        os.rename('models/' + latest_model_file, 'models/' + model_file + '_v1')

transformer_multivariate_quarter_hour_vis_columns_v1.csv
transformer_multivariate_quarter_hour_tmp_model_v1.pth
transformer_multivariate_half_hour_tmp_columns_v1.csv
transformer_multivariate_hour_vis_columns_v1.csv
transformer_model_tmp_v1.pth
transformer_multivariate_hour_hum_model_v1.pth
.DS_v1
renaming .DS_v1 to .DS_v1.DS_v1
transformer_multivariate_hour_vis_scaler_v1.pth
transformer_multivariate_hour_vis_model_v1.pth
transformer_multivariate_hour_tmp_columns_v1.csv
transformer_scaler_tmp_v1.pth
transformer_multivariate_half_hour_CO2_model_v1.pth
transformer_multivariate_half_hour_hum_scaler_v1.pth
transformer_multivariate_quarter_hour_CO2_model_v1.pth
transformer_multivariate_quarter_hour_hum_columns_v1.csv
transformer_multivariate_hour_hum_columns_v1.csv
transformer_multivariate_quarter_hour_CO2_scaler_v1.pth
transformer_multivariate_quarter_hour_vis_model_v1.pth
transformer_multivariate_hour_tmp_scaler_v1.pth
transformer_multivariate_half_hour_VOC_columns_v1.csv
transformer_multi

In [7]:
importlib.reload(utils)
model, scaler, rmse, mae, mape = utils.create_multivariate_transformer_model_for_feature(df, d_model=128, nhead=8, num_layers=4, dropout=0.1, batch_size=128, learning_rate=0.00025, epochs=2)

training data cutoff:  2023-07-15 02:45:00


/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:868: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_count = x[0].shape[1]
/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:869: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  print("feature count: ", feature_count)
/Users/timehmann/Library/Mob

tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday                     float64
month                       float64
hour_sin                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                       float64
device_id_hka-aqm-am001     float64
device_id_hka-aqm-am002     float64
device_id_hka-aqm-am003a    float64
device_id_hka-aqm-am003b    float64
device_id_hka-aqm-am004     

TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
importlib.reload(utils)
model, scaler = utils.create_multivariate_model_for_feature(df, aggregation_level='half_hour')

Training data shape: torch.Size([150050, 47]) torch.Size([150050])
Testing data shape: torch.Size([50023, 47]) torch.Size([50023])
     actual  predicted
0     408.0  477.67099
1     410.0  477.67099
2     406.5  477.67099
3     410.5  477.67099
4     413.5  477.67099
..      ...        ...
123   460.0  477.67099
124   440.0  477.67099
125   432.0  477.67099
126   432.5  477.67099
127   423.5  477.67099

[128 rows x 2 columns]
Epoch 1/1000, Validation Loss: 1.3423


KeyboardInterrupt: 

In [ ]:
train.columns

Index(['tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'WIFI', 'BLE', 'rssi',
       'channel_rssi', 'channel_index', 'spreading_factor', 'bandwidth',
       'f_cnt', 'isHoliday', 'isExamTime', 'weekday', 'month', 'hour_sin',
       'semester_SS23', 'semester_WS22/23', 'semester_WS23/24', 'tmp_lag_1',
       'tmp_lag_2', 'tmp_lag_3', 'tmp_lag_4', 'tmp_lag_5', 'hum_lag_1',
       'hum_lag_2', 'hum_lag_3', 'hum_lag_4', 'hum_lag_5', 'CO2_lag_1',
       'CO2_lag_2', 'CO2_lag_3', 'CO2_lag_4', 'CO2_lag_5', 'CO2_next',
       'VOC_lag_1', 'VOC_lag_2', 'VOC_lag_3', 'VOC_lag_4', 'VOC_lag_5',
       'vis_lag_1', 'vis_lag_2', 'vis_lag_3', 'vis_lag_4', 'vis_lag_5',
       'CO2_next_scaled'],
      dtype='object')

In [50]:
from datetime import datetime

importlib.reload(utils)
start_time = datetime.strptime('2022-11-03 10:00:00', '%Y-%m-%d %H:%M:%S')
predictions = utils.predict_data_multivariate_transformer(start_time=start_time, selected_room='am001', prediction_count=5)
predictions

loading latest scaler: models/transformer_multivariate_quarter_hour_CO2_scaler_v1.pth
loading latest model: models/transformer_multivariate_quarter_hour_CO2_model_v1.pth


/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:409: UserWarning: X does not have valid f

,date_time_rounded,CO2_pred
0,2022-11-03 00:00:00,458.0
1,2022-11-03 00:15:00,457.0
2,2022-11-03 00:30:00,463.0
3,2022-11-03 00:45:00,462.0
4,2022-11-03 01:00:00,459.0
...,...,...
91,2022-11-03 22:45:00,461.0
92,2022-11-03 23:00:00,461.0
93,2022-11-03 23:15:00,461.0
94,2022-11-03 23:30:00,461.0


In [21]:
predictions.columns

Index(['date_time_rounded', 'tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'WIFI',
       'BLE', 'rssi', 'channel_rssi', 'channel_index', 'spreading_factor',
       'bandwidth', 'f_cnt', 'isHoliday', 'isExamTime', 'weekday', 'month',
       'hour_sin', 'semester_SS23', 'semester_WS22/23', 'semester_WS23/24',
       'group', 'device_id_hka-aqm-am001', 'device_id_hka-aqm-am002',
       'device_id_hka-aqm-am003a', 'device_id_hka-aqm-am003b',
       'device_id_hka-aqm-am004', 'device_id_hka-aqm-am005',
       'device_id_hka-aqm-am107', 'device_id_hka-aqm-am109',
       'device_id_hka-aqm-am110', 'device_id_hka-aqm-am111',
       'device_id_hka-aqm-am115', 'device_id_hka-aqm-am116',
       'device_id_hka-aqm-am117', 'device_id_hka-aqm-am123',
       'device_id_hka-aqm-am124', 'device_id_hka-aqm-am126',
       'device_id_hka-aqm-am201a', 'device_id_hka-aqm-am201b',
       'device_id_hka-aqm-am204', 'device_id_hka-aqm-am205',
       'device_id_hka-aqm-am209', 'device_id_hka-aqm-am210',
       'devi